<a href="https://colab.research.google.com/github/RepnaiaE/DataScience_Emotion-recognition-methods/blob/main/Lab3_Extracting_features_from_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Извлечение признаков из текста/Extracting features from text

## 1 Загрузка датасета/Loading dataset

In [ ]:
import pandas as pd

negative = pd.read_csv('/content/drive/MyDrive/Распознавание эмоций/negative.csv', sep=';',
                       names=['id', 'tdate', 'tname', 'ttext', 'ttype', 'trep', 'trtv', 'tfav', 'tstcount', 'tfol', 'tfrien', 'listcount'])
negative.head()

,id,tdate,tname,ttext,ttype,trep,trtv,tfav,tstcount,tfol,tfrien,listcount
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0


In [ ]:
positive = pd.read_csv('/content/drive/MyDrive/Распознавание эмоций/positive.csv', sep=';',
                       names=['id', 'tdate', 'tname', 'ttext', 'ttype', 'trep', 'trtv', 'tfav', 'tstcount', 'tfol', 'tfrien', 'listcount'])
positive.head()

,id,tdate,tname,ttext,ttype,trep,trtv,tfav,tstcount,tfol,tfrien,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


Объединение корпусов по 1000 записей из каждого в один датасет

In [ ]:
concated_corp = pd.concat([positive[:1000], negative[:1000]])
concated_corp.shape

(2000, 12)

## 2 Удаление стоп слов/Removing stop words

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('russian'))
stop_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'а',
 'без',
 'более',
 'больше',
 'будет',
 'будто',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вас',
 'вдруг',
 'ведь',
 'во',
 'вот',
 'впрочем',
 'все',
 'всегда',
 'всего',
 'всех',
 'всю',
 'вы',
 'где',
 'да',
 'даже',
 'два',
 'для',
 'до',
 'другой',
 'его',
 'ее',
 'ей',
 'ему',
 'если',
 'есть',
 'еще',
 'ж',
 'же',
 'за',
 'зачем',
 'здесь',
 'и',
 'из',
 'или',
 'им',
 'иногда',
 'их',
 'к',
 'как',
 'какая',
 'какой',
 'когда',
 'конечно',
 'кто',
 'куда',
 'ли',
 'лучше',
 'между',
 'меня',
 'мне',
 'много',
 'может',
 'можно',
 'мой',
 'моя',
 'мы',
 'на',
 'над',
 'надо',
 'наконец',
 'нас',
 'не',
 'него',
 'нее',
 'ней',
 'нельзя',
 'нет',
 'ни',
 'нибудь',
 'никогда',
 'ним',
 'них',
 'ничего',
 'но',
 'ну',
 'о',
 'об',
 'один',
 'он',
 'она',
 'они',
 'опять',
 'от',
 'перед',
 'по',
 'под',
 'после',
 'потом',
 'потому',
 'почти',
 'при',
 'про',
 'раз',
 'разве',
 'с',
 'сам',
 'свою',
 'себе',
 'себя',
 'сейчас',
 'со',
 'совсем',
 'так

In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def del_stop_words(text):
    tokens = word_tokenize(text)
    processed = []
    for token in tokens:
        if token != '' and token not in stop_words:
            processed.append(token)

    return processed

concated_corp['tokenized'] = [del_stop_words(text) for text in concated_corp.ttext]
concated_corp[['ttext', 'tokenized']].head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,ttext,tokenized
0,"@first_timee хоть я и школота, но поверь, у на...","[@, first_timee, школота, ,, поверь, ,, самое,..."
1,"Да, все-таки он немного похож на него. Но мой ...","[Да, ,, все-таки, немного, похож, ., Но, мальч..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,"[RT, @, KatiaCheh, :, Ну, идиотка, ), испугала..."
3,"RT @digger2912: ""Кто то в углу сидит и погибае...","[RT, @, digger2912, :, ``, Кто, углу, сидит, п..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...,"[@, irina_dyshkant, Вот, значит, страшилка, :,..."


## 3 Извлечение лингвистических признаков/Extraction of linguistic features

In [ ]:
!pip install pymorphy2

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

Проводим удаление стоп слов из предыдущего пункта

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')


tokenizer = RegexpTokenizer(r'\w+')

def del_stop_words(text):
    tokens = tokenizer.tokenize(text.lower())
    processed = []
    for token in tokens:
        token = morph.parse(token)[0].normal_form
        if token != '' and token not in stop_words:
            processed.append(token)

    return processed

concated_corp['tokenized'] = [del_stop_words(text) for text in concated_corp.ttext]
concated_corp[['ttext', 'tokenized']].head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,ttext,tokenized
0,"@first_timee хоть я и школота, но поверь, у на...","[first_timee, школотый, поверь, самый, d, обще..."
1,"Да, все-таки он немного похож на него. Но мой ...","[всё, таки, немного, похожий, мальчик, всё, ра..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,"[rt, katiacheh, идиотка, испугаться]"
3,"RT @digger2912: ""Кто то в углу сидит и погибае...","[rt, digger2912, угол, сидеть, погибать, голод..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...,"[irina_dyshkant, значит, страшилка, d, блин, п..."


Загрузка корпуса linis.

In [ ]:
linis = pd.read_csv('/content/drive/MyDrive/Распознавание эмоций/dictionary_documents/words_all_full_rating.csv', sep=';',
                    encoding='windows-1251', engine='python')
linis.head()

,Words,mean,dispersion,average rate,Unnamed: 4
0,аборигенный,"-0,25","0,433012701892219",0,NaN
1,аборт,-1,"0,816496580927726",-1,NaN
2,абрамович,0,0,0,NaN
3,абсолютный,"0,333333333333333","0,471404520791032",0,NaN
4,абстрактный,"-0,111111111111111","0,87488976377909",0,NaN


Для каждого слова из объединенного корпуса, найденого в linis, сохранить его значение в список, характеризующий данную запись.

In [ ]:
all_rates = []

for tokens in concated_corp.tokenized:
    list_of_rates = []
    for token in tokens:
        row = linis.loc[linis['Words'] == token]
        if len(row) != 0:
            list_of_rates.append(row['mean'].values[0])

    if len(list_of_rates) == 0:
        list_of_rates.append(0)

    all_rates.append(list_of_rates)

concated_corp['rates'] = all_rates
concated_corp[['tokenized', 'rates']].head(10)

,tokenized,rates
0,"[first_timee, школотый, поверь, самый, d, обще...","[0,333333333333333, 0, 0, 0]"
1,"[всё, таки, немного, похожий, мальчик, всё, ра...","[0, 1]"
2,"[rt, katiacheh, идиотка, испугаться]","[-0,833333333333333]"
3,"[rt, digger2912, угол, сидеть, погибать, голод...","[-0,2, -1,33333333333333, 0]"
4,"[irina_dyshkant, значит, страшилка, d, блин, п...","[0,333333333333333, 0]"
5,"[любить, знать, бля, d, http, t, co, brf9eng1u6]",[0]
6,"[rt, spoonlamer, ох, 900, d, это, twinkleayo, ...","[0,333333333333333]"
7,"[rt, veregijytaqo, ухажёр, ухо, жрать, d]",[0]
8,"[поприветствовать, новый, читатель, alexey1789]",[0]
9,"[частичка, сидней, sydney, sydneyoperahouse, h...",[0]


Из списка значений создается вектор из 6 значений: mean, max, min, sum, sum_positive, sum_negative

In [ ]:
import numpy as np

features = []

for rate in concated_corp.rates:
    feature = []

    rate = [float(r.replace(',', '.')) if type(r) == str else float(r) for r in rate]

    feature.append(np.mean(rate))
    feature.append(np.max(rate))
    feature.append(np.min(rate))
    feature.append(np.sum(rate))
    feature.append(np.sum([1 if r > 0 else 0 for r in rate]))
    feature.append(np.sum([1 if r < 0 else 0 for r in rate]))

    features.append(feature)

concated_corp['features'] = features

concated_corp[['tokenized', 'rates', 'features']].head(10)

,tokenized,rates,features
0,"[first_timee, школотый, поверь, самый, d, обще...","[0,333333333333333, 0, 0, 0]","[0.08333333333333325, 0.333333333333333, 0.0, ..."
1,"[всё, таки, немного, похожий, мальчик, всё, ра...","[0, 1]","[0.5, 1.0, 0.0, 1.0, 1, 0]"
2,"[rt, katiacheh, идиотка, испугаться]","[-0,833333333333333]","[-0.833333333333333, -0.833333333333333, -0.83..."
3,"[rt, digger2912, угол, сидеть, погибать, голод...","[-0,2, -1,33333333333333, 0]","[-0.51111111111111, 0.0, -1.33333333333333, -1..."
4,"[irina_dyshkant, значит, страшилка, d, блин, п...","[0,333333333333333, 0]","[0.1666666666666665, 0.333333333333333, 0.0, 0..."
5,"[любить, знать, бля, d, http, t, co, brf9eng1u6]",[0],"[0.0, 0.0, 0.0, 0.0, 0, 0]"
6,"[rt, spoonlamer, ох, 900, d, это, twinkleayo, ...","[0,333333333333333]","[0.333333333333333, 0.333333333333333, 0.33333..."
7,"[rt, veregijytaqo, ухажёр, ухо, жрать, d]",[0],"[0.0, 0.0, 0.0, 0.0, 0, 0]"
8,"[поприветствовать, новый, читатель, alexey1789]",[0],"[0.0, 0.0, 0.0, 0.0, 0, 0]"
9,"[частичка, сидней, sydney, sydneyoperahouse, h...",[0],"[0.0, 0.0, 0.0, 0.0, 0, 0]"


In [ ]:
concated_corp.to_csv('/content/drive/MyDrive/Распознавание эмоций/ru_tweet_corp_ling_features.csv')

## 4 Извлечение морфологических признаков/Extraction of morphological features

In [ ]:
!pip install pymorphy2
!pip install pymorphy2-dicts
!pip install DAWG-Python

Определение части речи для каждого слова в предложениях. Для каждой записи подсчитывается количество каждой найденной части речи/Determine the part of speech for each word in sentences. For each entry, the number of each found part of speech is counted.

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')

from collections import defaultdict


tokenizer = RegexpTokenizer(r'\w+')

def get_poses_for_tokens(text):
    tokens = tokenizer.tokenize(text.lower())
    processed_count = defaultdict(int)
    for token in tokens:
        pos = morph.parse(token)[0].tag.POS
        if pos != None:
            processed_count[pos] += 1

    return processed_count

concated_corp['pos_count'] = [get_poses_for_tokens(text) for text in concated_corp.ttext]
concated_corp[['ttext', 'tokenized', 'pos_count']].head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,ttext,tokenized,pos_count
0,"@first_timee хоть я и школота, но поверь, у на...","[first_timee, школотый, поверь, самый, d, обще...","{'PRCL': 2, 'NPRO': 2, 'CONJ': 4, 'ADJS': 1, '..."
1,"Да, все-таки он немного похож на него. Но мой ...","[всё, таки, немного, похожий, мальчик, всё, ра...","{'PRCL': 4, 'NPRO': 2, 'ADVB': 1, 'ADJS': 1, '..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,"[rt, katiacheh, идиотка, испугаться]","{'PRCL': 1, 'NPRO': 3, 'NOUN': 1, 'VERB': 1, '..."
3,"RT @digger2912: ""Кто то в углу сидит и погибае...","[rt, digger2912, угол, сидеть, погибать, голод...","{'NPRO': 2, 'CONJ': 6, 'PREP': 2, 'NOUN': 3, '..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...,"[irina_dyshkant, значит, страшилка, d, блин, п...","{'PRCL': 2, 'CONJ': 6, 'NOUN': 4, 'INTJ': 1, '..."


Подсчитываем относительную частоту встречаемости частей речи в записях

In [ ]:
all_parts = defaultdict(int)
processed_float_values = []

for poses in concated_corp['pos_count']:
    processed = defaultdict(float)
    summ = np.sum([v for _, v in poses.items()])
    for k, v in poses.items():
        all_parts[k] += v
        processed[k] = v/summ

    processed_float_values.append(processed)

concated_corp['pos'] = processed_float_values
concated_corp[['ttext', 'tokenized', 'pos_count', 'pos']].head()

,ttext,tokenized,pos_count,pos
0,"@first_timee хоть я и школота, но поверь, у на...","[first_timee, школотый, поверь, самый, d, обще...","{'PRCL': 2, 'NPRO': 2, 'CONJ': 4, 'ADJS': 1, '...","{'PRCL': 0.13333333333333333, 'NPRO': 0.133333..."
1,"Да, все-таки он немного похож на него. Но мой ...","[всё, таки, немного, похожий, мальчик, всё, ра...","{'PRCL': 4, 'NPRO': 2, 'ADVB': 1, 'ADJS': 1, '...","{'PRCL': 0.2857142857142857, 'NPRO': 0.1428571..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,"[rt, katiacheh, идиотка, испугаться]","{'PRCL': 1, 'NPRO': 3, 'NOUN': 1, 'VERB': 1, '...","{'PRCL': 0.14285714285714285, 'NPRO': 0.428571..."
3,"RT @digger2912: ""Кто то в углу сидит и погибае...","[rt, digger2912, угол, сидеть, погибать, голод...","{'NPRO': 2, 'CONJ': 6, 'PREP': 2, 'NOUN': 3, '...","{'NPRO': 0.09523809523809523, 'CONJ': 0.285714..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...,"[irina_dyshkant, значит, страшилка, d, блин, п...","{'PRCL': 2, 'CONJ': 6, 'NOUN': 4, 'INTJ': 1, '...","{'PRCL': 0.1111111111111111, 'CONJ': 0.3333333..."


Итоговые вектора pos_count не одинаковы, так как в разных записях встречались различные части речи. Дополняем записи нулевыми значениями в одинаковом порядке.

In [ ]:
formatted_morph_features = []
for poses in concated_corp['pos']:
    for expected, _ in all_parts.items():
        if expected not in poses:
            poses[expected] = 0.0

    formatted_morph_features.append(poses)

concated_corp['all_pos'] = formatted_morph_features
concated_corp[['tokenized', 'pos_count', 'pos', 'all_pos']].head()

,tokenized,pos_count,pos,all_pos
0,"[first_timee, школотый, поверь, самый, d, обще...","{'PRCL': 2, 'NPRO': 2, 'CONJ': 4, 'ADJS': 1, '...","{'PRCL': 0.13333333333333333, 'NPRO': 0.133333...","{'PRCL': 0.13333333333333333, 'NPRO': 0.133333..."
1,"[всё, таки, немного, похожий, мальчик, всё, ра...","{'PRCL': 4, 'NPRO': 2, 'ADVB': 1, 'ADJS': 1, '...","{'PRCL': 0.2857142857142857, 'NPRO': 0.1428571...","{'PRCL': 0.2857142857142857, 'NPRO': 0.1428571..."
2,"[rt, katiacheh, идиотка, испугаться]","{'PRCL': 1, 'NPRO': 3, 'NOUN': 1, 'VERB': 1, '...","{'PRCL': 0.14285714285714285, 'NPRO': 0.428571...","{'PRCL': 0.14285714285714285, 'NPRO': 0.428571..."
3,"[rt, digger2912, угол, сидеть, погибать, голод...","{'NPRO': 2, 'CONJ': 6, 'PREP': 2, 'NOUN': 3, '...","{'NPRO': 0.09523809523809523, 'CONJ': 0.285714...","{'NPRO': 0.09523809523809523, 'CONJ': 0.285714..."
4,"[irina_dyshkant, значит, страшилка, d, блин, п...","{'PRCL': 2, 'CONJ': 6, 'NOUN': 4, 'INTJ': 1, '...","{'PRCL': 0.1111111111111111, 'CONJ': 0.3333333...","{'PRCL': 0.1111111111111111, 'CONJ': 0.3333333..."


In [ ]:
sorted_all_parts = sorted(all_parts.items(), key=lambda item: item[1])
type(sorted_all_parts)

list

Части речи сохранены в виде словаря, закодируем их в вектор в определенной очередности, одинаковой для всех

In [ ]:
def code_speech_parts(poses):
    code = []
    for part, _ in sorted_all_parts:
        code.append(poses[part])

    return code

concated_corp['coded_pos'] = [code_speech_parts(pose) for pose in concated_corp['all_pos']]
concated_corp[['tokenized', 'all_pos', 'coded_pos']].head()

,tokenized,all_pos,coded_pos
0,"[first_timee, школотый, поверь, самый, d, обще...","{'PRCL': 0.13333333333333333, 'NPRO': 0.133333...","[0.0, 0.0, 0.0, 0.06666666666666667, 0.0, 0.06..."
1,"[всё, таки, немного, похожий, мальчик, всё, ра...","{'PRCL': 0.2857142857142857, 'NPRO': 0.1428571...","[0.0, 0.0, 0.0, 0.0, 0.07142857142857142, 0.07..."
2,"[rt, katiacheh, идиотка, испугаться]","{'PRCL': 0.14285714285714285, 'NPRO': 0.428571...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[rt, digger2912, угол, сидеть, погибать, голод...","{'NPRO': 0.09523809523809523, 'CONJ': 0.285714...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.047..."
4,"[irina_dyshkant, значит, страшилка, d, блин, п...","{'PRCL': 0.1111111111111111, 'CONJ': 0.3333333...","[0.0, 0.05555555555555555, 0.0, 0.0, 0.0, 0.0,..."


In [ ]:
concated_corp.to_csv('/content/drive/MyDrive/Распознавание эмоций/ru_tweet_corp_morph_features.csv')

## 5 TF-IDF

In [ ]:
import sklearn

vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(concated_corp.ttext.to_list())
print(X.shape)

(2000, 1000)


In [ ]:
concated_corp['tfidf'] = X.toarray().tolist()
concated_corp[['tokenized', 'all_pos', 'tfidf']].head()

,tokenized,all_pos,tfidf
0,"[first_timee, школотый, поверь, самый, d, обще...","{'PRCL': 0.13333333333333333, 'NPRO': 0.133333...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[всё, таки, немного, похожий, мальчик, всё, ра...","{'PRCL': 0.2857142857142857, 'NPRO': 0.1428571...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[rt, katiacheh, идиотка, испугаться]","{'PRCL': 0.14285714285714285, 'NPRO': 0.428571...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[rt, digger2912, угол, сидеть, погибать, голод...","{'NPRO': 0.09523809523809523, 'CONJ': 0.285714...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[irina_dyshkant, значит, страшилка, d, блин, п...","{'PRCL': 0.1111111111111111, 'CONJ': 0.3333333...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
concated_corp.to_csv('/content/drive/MyDrive/Распознавание эмоций/ru_tweet_corp_tfidf.csv')

# Вывод/Conclusion

В данной лабораторной работе были рассмотрены способы представления признаков текста и способы их выделения. Выделены 3 набора признаков, которые будут в дальнейшем использоваться для обучения моделей машинного обучения:
* Лингвистические признаки
* Морфологичекские признаки
* TF-IDF

In this laboratory work, ways of presenting text features and ways to highlight them were considered. Three sets of features have been identified that will be further used to train machine learning models:

Linguistic features
Morphological features
TF-IDF